In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [14]:

# Load models
model_paths = [
    ("/content/drive/MyDrive/FYP/combine_model/mobilenet/1.keras", (256, 256), "mobilenet"),
    ("/content/drive/MyDrive/FYP/combine_model/nasnetMobile/1.keras", (256, 256), "nasnetmobile"),
    ("/content/drive/MyDrive/FYP/combine_model/vgg/2.keras", (256, 256), "vgg")  # VGG input size is 256
]

models = [load_model(path) for path, _, _ in model_paths]


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
#Test data path
test_data_dir = "/content/drive/MyDrive/FYP/split/test"

# Create an ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(256,256),  # Use the appropriate input size for your models
    batch_size=32,  # Adjust as needed
    class_mode='categorical',
    shuffle=False #Important for getting correct predictions
)


Found 131 images belonging to 2 classes.


In [16]:
# Predict using each model
predictions = []
for model, (path, input_size, model_name) in zip(models, model_paths): # Corrected: Iterating through model_paths to get all elements
    img_width, img_height = input_size  # Unpack input_size here

    print(f"Evaluating {model_name} model...")

    #Check if the model input matches the image size of the test generator
    if (img_width, img_height) != test_generator.image_shape[:2]:
        print(f"Warning: Model {model_name} input size ({img_width}, {img_height}) does not match test generator image shape {test_generator.image_shape[:2]}. Predictions might be incorrect.")

    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    predictions.append((model_name, y_pred_classes))

# Get true labels
y_true = test_generator.classes

# Calculate and print accuracy for each model
for model_name, y_pred_classes in predictions:
    accuracy = accuracy_score(y_true, y_pred_classes)
    print(f"Accuracy for {model_name}: {accuracy}")

Evaluating mobilenet model...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 97s 18s/step
Evaluating nasnetmobile model...
5/5 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step
Evaluating vgg model...
5/5 ━━━━━━━━━━━━━━━━━━━━ 114s 21s/step
Accuracy for mobilenet: 0.9007633587786259
Accuracy for nasnetmobile: 0.8702290076335878
Accuracy for vgg: 0.8549618320610687


In [17]:
# prompt: apply softvoting on each these models ensemble learning

from scipy.stats import mode

# Soft Voting
soft_predictions = np.array([model.predict(test_generator) for model in models])
soft_predictions = np.mean(soft_predictions, axis=0)
soft_predicted_classes = np.argmax(soft_predictions, axis=1)

# Calculate and print accuracy for soft voting
soft_accuracy = accuracy_score(y_true, soft_predicted_classes)
print(f"Accuracy for Soft Voting: {soft_accuracy}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 91s 16s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 112s 21s/step
Accuracy for Soft Voting: 0.9236641221374046
